# GEE Python API Demo

#### Reworked a TON of stuff, a few highlights:
- Started building out a libary of handy functions (especially those that you'd need for automating data gathering
- Created a bit of documentation but that takes ages
- Still working on a required "install list", which is currently saved as a comment in the first cell of the Setup section 

## Setup

### Imports

In [1]:
#pip install earthengine-api

import ee
import json
import ee.mapclient
import pandas as pd
import numpy as np

### Setting up Google Earth

In [2]:
ee.Authenticate()

Enter verification code:  4/2wE-JnqlkPAOWwutJJpf5AHPT9PA0-1biKFWKkli9fkGY5IDOEfrhh8



Successfully saved authorization token.


In [3]:
ee.Initialize()

### Various methods, just initialize then hide them

In [55]:
def printImgInfo(img,formatted=True,returnstr=True,compact=False,sort_k=False):
    """
    Calls .imgInfo() (creates a str using JSON conventions) and prints to console 
    
    Parameters: 
    img (ee.Image): image to get info from
    formatted (Boolean=True): prints out using standard "pretty JSON" conventions, tab=2 spaces
    returnstr (Boolean=True): set false if you don't need to store the results
    compact (Boolean=False): crams everything onto one line with as little whitespace as possible 
    sort_k (Boolean=False): sorts the keys of the JSON (unintuitive results with nested lists)
    
    Returns: 
    str: JSON formatted string from JSON library's .dumps() method
    
    """
    
    img_info_json = imgInfo(img,formatted,compact, sort_k)
    print(img_info_json)
    if returnstr: return img_info_json

def imgInfo(img,formatted=True,compact=False, sort_k=False):
    """
    Returns a JSON formatted string of properties/metadata for img.
    
    Parameters: 
        img (ee.Image): image to get info from
        formatted (Boolean=True): use standard "pretty JSON" conventions, (tab=2 spaces)
        compact (Boolean=False): crams everything onto one line and cuts all extra whitespace
            Default behavior is 'formatted',
            BUT Precedence is 'compact' > 'formatted' > unformatted (one line but w/ whitespace)
            
                Compact is checked first, otherwise you'd have to specify both each time
                (based on my default args, so it could be changed if it's annoying)
                
        sort_k (Boolean=False): sorts the JSON's keys (unintuitive results with nested lists)
    
    Returns: 
        str: JSON formatted string from JSON library's .dumps() method
    """
    if compact: return json.dumps(img.getInfo(),indent=None,sort_keys=sort_k)
    elif formatted: return json.dumps(img.getInfo(),indent=2,sort_keys=sort_k)
    else: return json.dumps(img.getInfo(),indent=0,sort_keys=sort_k)

def print_rect_info(vertices):
    if not (len(vertices) == 4):
        print("error: pass in the correct number of vertices, provided: ")
        print(vertices)
        return
    
    print(rectlist_to_str(vertices))
    # for v in vertices: print(v)
    
    x_length = max(vertices[0][0],vertices[1][0],vertices[2][0],vertices[3][0]) - min(vertices[0][0],vertices[1][0],vertices[2][0],vertices[3][0])
    y_length = max(vertices[0][1],vertices[1][1],vertices[2][1],vertices[3][1]) - min(vertices[0][1],vertices[1][1],vertices[2][1],vertices[3][1])
    
    print("latitude spanned: "+str(y_length))
    print("longitude spanned: "+str(x_length))
    
    angle_offset = ((np.arctan2((vertices[1][1]-vertices[0][1]),(vertices[1][0]-vertices[0][0])))*180)/(np.pi)
    
    print("angle offset between p1 and p2 (from x-axis/equator): " + str(angle_offset)+ " (degrees: " + str(angle_offset) +")")

def rectlist_to_str(rect,sort_v=False):
    if sort_v:
        wtf = pd.DataFrame({'x': [rect[0][0],
                                rect[1][0],
                                rect[2][0],
                                rect[3][0] ] ,
                            'y': [rect[0][1],
                                rect[1][1],
                                rect[2][1],
                                rect[3][1] ] })
        wtf.sort_values(by=['y','x'],inplace=True)
        wtf.reset_index(drop=True, inplace=True)
        rect = [ [ wtf['x'][i], wtf['y'][i] ] for i in range(4)]
    return str('[' + str(rect[0]) + ', ' + str(rect[1]) + ', ' + str(rect[2]) + ', ' + str(rect[3]) + ']')

def region_rect_pointlen(x, y, len_x=0, len_y=0, rad=0):
    new_v = []
    if not (rad == 0): 
        len_x = abs(rad)*2
        len_y = abs(rad)*2
        x -= rad
        y -= rad
    elif (len_x == 0) & (len_y == 0):  
        len_x = 1
        len_y = 1
        
    return [[ x         , y         ],
            [ x + len_x , y         ],
            [ x         , y + len_y ],
            [ x + len_x , y + len_y ] ]

def region_rect_list(vertices,len_x=0,len_y=0): 
    # assumes it's always oriented NSEQ. Must have 2 vertices, 
    # either they're collinear or "diagonal"
    # list of [x,y], so for kth vertex, (vertices[k])
    # x coord = vertices[k][0] and y coord = vertices[k][1] 
    
    if not len(vertices) == 2:
        print("error: pass in the correct number of vertices, returning random example rectanble")
        return [ [-120, 35] , [-119, 35] , [-119, 34] , [-120, 34] ]
    
    new_v = vertices.copy()
    
    if (len_x == 0) & (len_y == 0):  
        len_x = 1
        len_y = 1
        
    if not (vertices[0][0] == vertices[1][0]) & (vertices[0][1] == vertices[1][1]):
        new_v.append([ vertices[0][0], vertices[1][1] ])
        new_v.append([ vertices[1][0], vertices[0][1] ])
        return new_v
    else:
        new_v.append([ vertices[0][0]+len_x , vertices[0][1]+len_y ])
        new_v.append([ vertices[1][0]+len_x , vertices[1][1]+len_y ])
        return new_v
        
def region_rect_twopoints(x1, y1, x2, y2, len_x=0, len_y=0):
    return region_rect_list([[x1,y1],[x2,y2]],len_x,len_y)

def getURL(img, region, scale=30, print_url=True,ret_as_str=False):
    if not type(region) == 'str': region = rectlist_to_str(region)
    
    url = img.getDownloadUrl({
                    'scale': 30,
                    'crs': 'EPSG:4326',
                    'region': '[[-120, 35], [-119, 35], [-119, 34], [-120, 34]]' 
                    })
    
    if print_url: print(str(url))
    
    if ret_as_str: return str(url)
    else: return url

## Some demonstrations

In [59]:
# Load a Landsat image.
img1 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_034033_20000913')
# img_info = printImgInfo(img1) # or try out img_info = imgInfo(img1)


# target_region = '[[-120, 35], [-119, 35], [-119, 34], [-120, 34]]'
target_region = region_rect_pointlen(-119,34,len_x=1, len_y=1,rad=0)
print('Target region:')
print_rect_info(target_region) 
print()

print('region_rect_pointlen: ' + rectlist_to_str(region_rect_pointlen(-119.5,34.5,rad=1)))
print('region_rect_list: ' + rectlist_to_str(region_rect_list([[-119, 34], [-119, 35]], len_x=1))) 
#also accepts len_x,len_y, but no radius. Defaultlengths are 0 but it sets them to 1 if they're both 0) 
print('region_rect_list: ' + rectlist_to_str(region_rect_list([[-119, 34], [-120, 35]])))
print('region_rect_twopoints: ' + rectlist_to_str(region_rect_twopoints(-119, 34, -120, 35))) #basically just calls region_rect_list but builds the list for you
print('region_rect_twopoints: ' + rectlist_to_str(region_rect_twopoints(-119, 34, -119, 35, len_x=1)))
print()

img2 = ee.Image('CGIAR/SRTM90_V4')
print('image url function: ')
url = getURL(img2, target_region, scale=30, print_url=True,ret_as_str=False)

Target region:
[[-119, 34], [-118, 34], [-119, 35], [-118, 35]]
latitude spanned: 1
longitude spanned: 1
angle offset between p1 and p2 (from x-axis/equator): 0.0 (degrees: 0.0)

region_rect_pointlen: [[-120.5, 33.5], [-118.5, 33.5], [-120.5, 35.5], [-118.5, 35.5]]
region_rect_list: [[-119, 34], [-119, 35], [-119, 35], [-119, 34]]
region_rect_list: [[-119, 34], [-120, 35], [-119, 35], [-120, 34]]
region_rect_twopoints: [[-119, 34], [-120, 35], [-119, 35], [-120, 34]]
region_rect_twopoints: [[-119, 34], [-119, 35], [-119, 35], [-119, 34]]

image url function: 
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/71d18b67c93051d02931c5cf15d2b857-38e998c371ad4b99c5aeec53b949649f:getPixels


## From this point onwards I haven't touched anything.

In [60]:
ee.Initialize()
ee.mapclient.centerMap(-107, 41, 6)

fc = ee.FeatureCollection([
    ee.Feature(
        ee.Geometry.Polygon(
            [[-120.16622848272344, 39.25150659029322],
          [-120.16622848272344, 38.93441896076937],
          [-119.91937942266485, 38.93441896076937],
          [-119.91937942266485, 39.25150659029322]])),
    ee.Feature(
        ee.Geometry.Polygon(
            [[-114.05, 37.0], [-109.05, 37.0], [-109.05, 41.0],
             [-111.05, 41.0], [-111.05, 42.0], [-114.05, 42.0]]),
        {'name': 'Utah', 'fill': 2})
    ])

# Fill, then outline the polygons into a blank image.
image1 = ee.Image(0).mask(0)
image2 = image1.paint(fc, 'fill')  # Get color from property named 'fill'
image3 = image2.paint(fc, 3, 5)    # Outline using color 3, width 5.
image4 = image3.toByte()

ee.mapclient.addToMap(image4, {
    'palette': ['000000', 'FF0000', '00FF00', '0000FF'],
    'max': 3,
    'opacity': 0.5
})



In [ ]:
# water function:
var waterfunction = function(image){
  //add the NDWI band to the image
  var ndwi = image.normalizedDifference(['B3', 'B5']).rename('NDWI');
  //get pixels above the threshold
  var water01 = ndwi.gt(waterThreshold);
  //mask those pixels from the image
  image = image.updateMask(water01).addBands(ndwi);

  var area = ee.Image.pixelArea();
  var waterArea = water01.multiply(area).rename('waterArea');

  image = image.addBands(waterArea);

  var stats = waterArea.reduceRegion({
    reducer: ee.Reducer.sum(), 
    geometry: geometry, 
    scale: 30,
  });

  return image.set(stats);
};
var collection = landsat8.map(waterfunction);
print(collection);

var chart = ui.Chart.image.series({
  imageCollection: collection.select('waterArea'), 
  region: geometry, 
  reducer: ee.Reducer.mean(), 
  scale: 30,
});
print(chart);


var landsat82 = ee.ImageCollection('LANDSAT/LC8_L1T_TOA').filterBounds(geometry);

var waterfunction2 = function(image){
  //add the NDWI band to the image
  var ndwi2 = image.normalizedDifference(['B3', 'B5']).rename('NDWI');
  var water02 = ndwi2.gt(waterThreshold+1);
  return image.updateMask(water02).addBands(ndwi2);
};

var col = landsat82.map(waterfunction2).select('NDWI');
var region = geometry;

col = col.map(function(img) {
  var doy = ee.Date(img.get('system:time_start')).getRelative('week', 'year');
  return img.set('doy', doy);
});

var distinctDOY = col.filterDate('2013-01-01', '2020-12-31');


var filter = ee.Filter.equals({leftField: 'doy', rightField: 'doy'});

var join = ee.Join.saveAll('doy_matches');

var joinCol = ee.ImageCollection(join.apply(distinctDOY, col, filter));

var comp = joinCol.map(function(img) {
  var doyCol = ee.ImageCollection.fromImages(
    img.get('doy_matches')
  );
  return doyCol.reduce(ee.Reducer.median());
});


var visParams = {
  palette: [
    'D4D0F6',	'CCC7F4',	'C3BEF3',	'BBB5F1',
'B1ACEF',	'A6A3EB',	'969AE6',	'8290DF',
'6F88D7',	'597DCC',	'4774C0',	'396BB1',
'3064A3',	'2A5C95',	'275186',	'27467A',
'283B6E',	'293265',	'2A295B',	'2B1E51',
]
};

var rgbVis = comp.map(function(img) {
  return img.visualize(visParams);
});

var gifParams = {
  'region': region,
  'dimensions': 300,
  'framesPerSecond': 12,
  'format': 'gif'
};

print(rgbVis.getVideoThumbURL(gifParams));

print(ui.Thumbnail(rgbVis, gifParams));

In [ ]:
# JavaScript to translate to Python

var geometry = 
    /* color: #ffc82d */
    /* displayProperties: [
      {
        "type": "rectangle"
      }
    ] */ 
    ee.Geometry.Polygon(
        [[[-120.16622848272344, 39.25150659029322],
          [-120.16622848272344, 38.93441896076937],
          [-119.91937942266485, 38.93441896076937],
          [-119.91937942266485, 39.25150659029322]]], null, false);
var landsat8= ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').filterBounds(geometry);
var waterThreshold = 0; 

// water function:
var waterfunction = function(image){
  //add the NDWI band to the image
  var ndwi = image.normalizedDifference(['B3', 'B5']).rename('NDWI');
  //get pixels above the threshold
  var water01 = ndwi.gt(waterThreshold);
  //mask those pixels from the image
  image = image.updateMask(water01).addBands(ndwi);

  var area = ee.Image.pixelArea();
  var waterArea = water01.multiply(area).rename('waterArea');

  image = image.addBands(waterArea);

  var stats = waterArea.reduceRegion({
    reducer: ee.Reducer.sum(), 
    geometry: geometry, 
    scale: 30,
  });

  return image.set(stats);
};
var collection = landsat8.map(waterfunction);
print(collection);

var chart = ui.Chart.image.series({
  imageCollection: collection.select('waterArea'), 
  region: geometry, 
  reducer: ee.Reducer.mean(), 
  scale: 30,
});
print(chart);


var landsat82 = ee.ImageCollection('LANDSAT/LC8_L1T_TOA').filterBounds(geometry);

var waterfunction2 = function(image){
  //add the NDWI band to the image
  var ndwi2 = image.normalizedDifference(['B3', 'B5']).rename('NDWI');
  var water02 = ndwi2.gt(waterThreshold+1);
  return image.updateMask(water02).addBands(ndwi2);
};

var col = landsat82.map(waterfunction2).select('NDWI');
var region = geometry;

col = col.map(function(img) {
  var doy = ee.Date(img.get('system:time_start')).getRelative('week', 'year');
  return img.set('doy', doy);
});

var distinctDOY = col.filterDate('2013-01-01', '2020-12-31');


var filter = ee.Filter.equals({leftField: 'doy', rightField: 'doy'});

var join = ee.Join.saveAll('doy_matches');

var joinCol = ee.ImageCollection(join.apply(distinctDOY, col, filter));

var comp = joinCol.map(function(img) {
  var doyCol = ee.ImageCollection.fromImages(
    img.get('doy_matches')
  );
  return doyCol.reduce(ee.Reducer.median());
});


var visParams = {
  palette: [
    'D4D0F6',	'CCC7F4',	'C3BEF3',	'BBB5F1',
'B1ACEF',	'A6A3EB',	'969AE6',	'8290DF',
'6F88D7',	'597DCC',	'4774C0',	'396BB1',
'3064A3',	'2A5C95',	'275186',	'27467A',
'283B6E',	'293265',	'2A295B',	'2B1E51',
]
};

var rgbVis = comp.map(function(img) {
  return img.visualize(visParams);
});

var gifParams = {
  'region': region,
  'dimensions': 300,
  'framesPerSecond': 12,
  'format': 'gif'
};

print(rgbVis.getVideoThumbURL(gifParams));

print(ui.Thumbnail(rgbVis, gifParams));